- 데이터 수집: https://www.federalregister.gov/developers/documentation/api/v1#/

# Baseline

In [7]:
import requests

API_BASE = "https://www.federalregister.gov/api/v1"
doc_num  = "2025-1000"
fields   = ["abstract", "title", "publication_date", "html_url"]

resp = requests.get(
    f"{API_BASE}/documents/{doc_num}.json",
    params={ "fields[]": fields }
)
resp.raise_for_status()

data = resp.json()
print("제목:", data.get("title"))
print("발행일:", data.get("publication_date"))
print("요약:", data.get("abstract"))
print("URL:", data.get("html_url"))


제목: Addition of Nicaragua to the List of Regions Affected With Screwworm
발행일: 2025-01-16
요약: We are advising the public that we have added Nicaragua to the Animal and Plant Health Inspection Service (APHIS) list maintained on the APHIS website of regions considered affected with screwworm. We took this action because of the confirmation of screwworm in this region.
URL: https://www.federalregister.gov/documents/2025/01/16/2025-01000/addition-of-nicaragua-to-the-list-of-regions-affected-with-screwworm


# 최신순

In [10]:
import requests

API_BASE = "https://www.federalregister.gov/api/v1"
params = {
    "order": "newest",            # 최신순
    "per_page": 1,                # 한 건만
    # 필요하다면 'conditions[...]' 으로 필터 추가
    "fields[]": [
        "document_number",
        "title",
        "publication_date",
        "abstract",
        "html_url",
    ],
}

# ✔ 검색용 엔드포인트 (/documents.json)
resp = requests.get(f"{API_BASE}/documents.json", params=params)
resp.raise_for_status()

data = resp.json()
print("응답에 포함된 키:", list(data.keys()))
# -> ['results', 'count', 'next_page_url', ...]

latest = data["results"][0]
print("가장 최근 문서:")
print(f"- 번호       : {latest['document_number']}")
print(f"- 제목       : {latest['title']}")
print(f"- 발행일     : {latest['publication_date']}")
print(f"- 요약(abstract): {latest.get('abstract')}")
print(f"- URL        : {latest['html_url']}")


응답에 포함된 키: ['count', 'description', 'total_pages', 'next_page_url', 'results']
가장 최근 문서:
- 번호       : 2025-13193
- 제목       : Amending the Administrative Procedures With Respect to the Import and Export of Natural Gas
- 발행일     : 2025-07-14
- 요약(abstract): The Department of Energy (DOE) is publishing this document to respond to comments received on the May 16, 2025, direct final rule. As a result, DOE delays the effective date of the direct final rule on the administrative procedures regarding the Office of Fossil Energy's (FE) filing requirements for the import and export of natural gas.
- URL        : https://www.federalregister.gov/documents/2025/07/14/2025-13193/amending-the-administrative-procedures-with-respect-to-the-import-and-export-of-natural-gas


# 특정 기간

In [12]:
import requests

API_BASE = "https://www.federalregister.gov/api/v1"
YEAR = "2025"

def fetch_all_docs_for_year(year: str):
    all_docs = []
    params = {
        # 연도 전체를 YYYY-MM-DD 범위로 지정
        "conditions[publication_date][]": f"{year}-01-01",
        "conditions[publication_date][]": f"{year}-12-31",
        "per_page": 100,       # 최대 100건씩
        "order": "newest",     # 최신순이든 상관없음
        "fields[]": [
            "document_number",
            "title",
            "publication_date",
            "html_url"
        ],
    }

    # 첫 페이지 URL
    url = f"{API_BASE}/documents.json"
    while url:
        resp = requests.get(url, params=params)
        resp.raise_for_status()
        data = resp.json()

        # 결과 누적
        all_docs.extend(data.get("results", []))

        # 다음 페이지로 이동 (params는 이후 무시됨)
        url = data.get("next_page_url")

        # 페이지네이션 파라미터는 처음 한 번만
        params = {}

    return all_docs


docs_2025 = fetch_all_docs_for_year(YEAR)
print(f"2025년 문서 총 {len(docs_2025)}건 수집 완료")
# 예시: 처음 5건 출력
for d in docs_2025[:5]:
    print(d["publication_date"], d["document_number"], d["title"])


2025년 문서 총 5000건 수집 완료
2025-07-14 2025-13193 Amending the Administrative Procedures With Respect to the Import and Export of Natural Gas
2025-07-14 2025-13173 Sunshine Act Meetings
2025-07-14 2025-13167 Sunshine Act Meetings
2025-07-14 2025-13166 Agency Information Collection Activities; Submission to the Office of Management and Budget (OMB) for Review and Approval; Comment Request; Surfclam/Ocean Quahog Individual Transferable Quota Administration
2025-07-14 2025-13165 Agency Information Collection Activities; Submission to the Office of Management and Budget (OMB) for Review and Approval; Comment Request; Alaska Region Bering Sea and Aleutian Islands Crab Economic Data Reports


# 연합뉴스 RSS
- https://www.yonhapnewstv.co.kr/add/rss

In [14]:
import feedparser

def fetch_yonhap_economy_news():
    feed_url = "http://www.yonhapnewstv.co.kr/category/news/economy/feed/"
    feed = feedparser.parse(feed_url)
    news_list = []
    for entry in feed.entries:
        news = {
            "title": entry.title,
            "link": entry.link,
            "published": entry.published,
            "summary": entry.summary
        }
        news_list.append(news)
    return news_list

# 예시: 최신 5개 경제 뉴스 출력
yonhap_news = fetch_yonhap_economy_news()
for news in yonhap_news[:5]:
    print(news["published"], news["title"])
    print(news["link"])
    print(news["summary"])
    print("-" * 80)

Mon, 14 Jul 2025 23:16:51 +0900 금감위? 금소원?…조직개편 장고에 정책 공백 우려
https://www.yonhapnewstv.co.kr/news/MYH20250714231633492
[앵커] 정부의 금융당국 조직 개편 논의가 길어지면서 새 금융당국 수장 임명도 늦어지고 있습니다. 원화 기반 스테이블코인 도입 등 현안이 산적한 가운데 정책 공백에 대한 우려가 나옵니다. 배시진 기자입니다. [기자] 정부가 지난주 19개 부처 장관 인선을 모두 마무리했지만, 금융당국 수장 지명 소식은 아직입니다. 대선 직후 사의를 표한 김병환 금융위원장의 후임이 정해지지 않았고, 김소영 전 금융위 부위원장과 이복현 전 금융감독원장 자리도 ...
--------------------------------------------------------------------------------
Mon, 14 Jul 2025 22:37:47 +0900 서울에서 바나나가?…폭염에 도심 한복판서 열매
https://www.yonhapnewstv.co.kr/news/MYH20250714223718994
[앵커] 올 여름 날씨가 심상치 않습니다. 역대급 폭염에 극한 폭우까지, 고온다습한 날씨가 이어지면서 서울 도심 한복판에서 바나나가 열리는 진풍경까지 벌어졌는데요. 기후변화로 국내에서도 아열대 과일 재배가 늘 거라는 전망이 나옵니다. 김도헌 기자가 바나나 농장에 다녀왔습니다. [기자] 커다란 이파리 사이로 초록색 바나나가 고개를 내밀었습니다. 평범한 외국산 바나나 같아 보이지만, 나무 뒤로 익숙한 아파트가 보입니다. 제가 나와 있는 이 곳...
--------------------------------------------------------------------------------
Mon, 14 Jul 2025 21:56:20 +0900 SK텔레콤, 위약금 면제 기간에 16만여명 이탈…KT·LGU+행
https://www.yonhapnewstv.co.kr/news/

# EDGAR
- https://api.edgarfiling.sec.gov/#tokens

In [ ]:
from dotenv import dotenv_values

# 수정된 API 키 로드 함수
def load_api_key_fixed():
    config = dotenv_values(dotenv_path="../.env")
    api_key = config.get('open_national_assembly_information')
    
    if api_key:
        print("국회 API 키가 성공적으로 로드되었습니다.")
        return api_key
    else:
        print("국회 API 키를 찾을 수 없습니다. .env 파일을 확인해주세요.")
        return None

# 견고한 예산안 데이터 수집 함수
def fetch_budget_bills_fixed(api_key, page_size=10, page_index=1):
    """
    국회 API의 다양한 응답 구조를 모두 처리할 수 있는 견고한 파싱 함수
    """
    base_url = "https://open.assembly.go.kr/portal/openapi/nwvrqwxyaytdsfvhu"
    
    params = {
        "KEY": api_key,
        "Type": "json",
        "pIndex": page_index,
        "pSize": page_size,
        "BILL_NAME": "예산"
    }
    
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        
        data = response.json()
        service_key = "nwvrqwxyaytdsfvhu"
        
        if service_key not in data:
            print(f"서비스 키 '{service_key}'가 응답에 없습니다.")
            print(f"사용 가능한 키들: {list(data.keys())}")
            return []
        
        service_data = data[service_key]
        
        def find_total_count(obj):
            """응답 구조에서 총 건수를 찾는 함수"""
            if isinstance(obj, dict):
                if "head" in obj:
                    head_info = obj["head"]
                    if isinstance(head_info, list) and len(head_info) > 0:
                        return head_info[0].get('list_total_count', 0)
                for value in obj.values():
                    result = find_total_count(value)
                    if result > 0:
                        return result
            elif isinstance(obj, list):
                for item in obj:
                    result = find_total_count(item)
                    if result > 0:
                        return result
            return 0
        
        def find_bills_data(obj):
            """응답 구조에서 실제 의안 데이터를 찾는 함수"""
            bills = []
            if isinstance(obj, dict):
                if "row" in obj:
                    row_data = obj["row"]
                    if isinstance(row_data, list):
                        for item in row_data:
                            if isinstance(item, dict) and "BILL_NAME" in item:
                                bills.append(item)
                for key, value in obj.items():
                    if key != "head":
                        bills.extend(find_bills_data(value))
            elif isinstance(obj, list):
                for item in obj:
                    bills.extend(find_bills_data(item))
            return bills
        
        # 총 건수 추출
        total_count = find_total_count(service_data)
        
        # 실제 데이터 추출
        bills = find_bills_data(service_data)
        
        # 중복 제거
        unique_bills = []
        seen_bill_nos = set()
        
        for bill in bills:
            if isinstance(bill, dict) and "BILL_NO" in bill:
                bill_no = bill["BILL_NO"]
                if bill_no not in seen_bill_nos:
                    unique_bills.append(bill)
                    seen_bill_nos.add(bill_no)
        
        print(f"총 건수: {total_count}")
        print(f"실제 반환된 데이터: {len(unique_bills)}건")
        
        return unique_bills
        
    except Exception as e:
        print(f"오류 발생: {e}")
        return []

# 테스트 실행
api_key = load_api_key_fixed()

if api_key:
    print("=== 수정된 함수로 예산안 데이터 수집 ===")
    budget_bills = fetch_budget_bills_fixed(api_key, page_size=10, page_index=1)
    
    if budget_bills:
        print(f"\n성공! {len(budget_bills)}건의 데이터를 수집했습니다.")
        
        # 처음 3개 항목 출력
        for i, bill in enumerate(budget_bills[:3], 1):
            print(f"\n=== 예산안 {i} ===")
            print(f"의안명: {bill.get('BILL_NAME', 'N/A')}")
            print(f"의안번호: {bill.get('BILL_NO', 'N/A')}")
            print(f"제안일자: {bill.get('PROPOSE_DT', 'N/A')}")
            print(f"제안자: {bill.get('PROPOSER', 'N/A')}")
            print(f"위원회: {bill.get('COMMITTEE', 'N/A')}")
            print(f"처리상태: {bill.get('PROC_RESULT', 'N/A')}")
            print(f"요약: {bill.get('SUMMARY', 'N/A')[:100]}...")
    else:
        print("데이터를 찾을 수 없습니다. API 키나 서비스 상태를 확인해주세요.")
else:
    print("API 키가 없어서 실행할 수 없습니다.")


In [ ]:
import requests

API_URL = "https://likms.assembly.go.kr/bill/billSearchListAjax.do"
params = {
    "searchType": "I",     # I: 입법예고
    "pageIndex": 1,        # 1페이지
    "pageUnit": 10,        # 한 번에 10건
    "orderBy": "REG_DT",   # 등록일자 순
}

resp = requests.get(API_URL, params=params)
resp.raise_for_status()
data = resp.json()

for item in data.get("list", []):
    print(f"- 제목: {item['billName']}")
    print(f"  발의번호: {item['billNo']}")
    print(f"  등록일: {item['regDateTime']}")
    print(f"  상세링크: https://likms.assembly.go.kr/bill/billDetail.do?billId={item['billId']}\n")


In [12]:
from dotenv import dotenv_values
config = dotenv_values(
    dotenv_path = "../.env"
)


In [13]:
config['open_national_assembly_information']

'284c7c0cdd934c36ad3e60094c11290a'